In [1]:
import spacy
import pickle
import neuralcoref.train.document as dc

In [2]:
with open('corefSpansObject', 'rb') as f:
    corefSpans = pickle.load(f)

In [3]:
sweCorp = ''

with open('../Data/Datasets/Europarl/Documents/ep-00-01-17/ep-00-01-17-sv.txt') as sweDoc:
    for line in sweDoc:
        sweCorp = sweCorp + line



In [4]:
nlp = spacy.load('../Models/SwedishModel')
doc = nlp(sweCorp)

In [5]:
corefSpansSwe = []
numOfCorefs = 0
for cluster in corefSpans:
    corefClusterSwe = []
    for mention in cluster:
        corefSwe = []
        startOfLine = 0
        endOfLine = 0
        lineText = ''
        mentionText = ''
        if mention[0] != -1:
            for tok in range(mention[0], mention[-1]):
                mentionText = mentionText + doc[tok].text + ' '
            mentionText.rstrip()

            startOfLine = mention[0]
            endOfLine = mention[-1]
            while doc[startOfLine].text != '\n':
                startOfLine -= 1
            startOfLine += 1
            while doc[endOfLine].text != '\n':
                endOfLine += 1

            for tok in range(startOfLine, endOfLine):
                lineText = lineText + doc[tok].text + ' '
            lineText.rstrip()

        mentionSpanDoc = nlp(mentionText)
        lineDoc = nlp(lineText)

        mentionsInSpan = dc.extract_mentions_spans(mentionSpanDoc, [])
        mentionsInLine = dc.extract_mentions_spans(lineDoc, [])

        if len(mentionsInSpan) == 0:
            continue

        invalidCoref = False
        
        """for span in mentionsInLine:
            if (span.end + startOfLine) > mention[1] > (span.start + startOfLine) > mention[0] or (span.start + startOfLine) < mention[0] < (span.end + startOfLine) < mention[1]:
                invalidCoref = True"""
        
        maxLength = 0
        maxSpan = 0
        for span in mentionsInSpan:
            if span.end - span.start > maxLength:
                maxLength = span.end - span.start
                maxSpan = span

        for span in mentionsInSpan:
            if span.start < maxSpan.start or span.end > maxSpan.end:
                invalidCoref = True

        if invalidCoref:
            continue
        elif not invalidCoref:
            corefSwe.append(maxSpan.start + mention[0])
            corefSwe.append(maxSpan.end + mention[0])
            corefClusterSwe.append(corefSwe)

    if len(corefClusterSwe) > 1:
        corefSpansSwe.append(corefClusterSwe)

In [6]:
noOfCorefsEng = 0
for sublist in corefSpans:
    for span in sublist:
        noOfCorefsEng += 1

In [7]:
noOfCorefsSwe = 0
for sublist in corefSpansSwe:
    for span in sublist:
        noOfCorefsSwe += 1

In [6]:
with open('corefSpansSweObject', 'wb') as f:
    pickle.dump(corefSpansSwe, f)
